In [23]:
import json
import re
#from collections import Counter
import pandas as pd

from pathlib import Path

In [25]:
# use flatten_claims.csv (filter out insufficient evidence, then group by claim, 1171 claims remain)
def summarize_claims_by_category_and_verdict(file_path, sep="\t"):

    df = pd.read_csv(file_path, sep=sep)

    claim_df = (
        df.groupby("claim_id", as_index=False)
        .agg({
            "factcheck_verdict": "first"   # safe by your data definition
        })
    )

    claim_df["category"] = claim_df["claim_id"].str.split(r"[-_]").str[0]
    claim_df["factcheck_verdict"] = claim_df["factcheck_verdict"].astype(str).str.strip()

    valid_verdicts = ["True", "False", "Half True"]

    verdict_counts = (
        claim_df[claim_df["factcheck_verdict"].isin(valid_verdicts)]
        .groupby(["category", "factcheck_verdict"])
        .size()
        .unstack(fill_value=0)
        .reindex(columns=valid_verdicts, fill_value=0)
    )

    totals = claim_df.groupby("category").size()
    verdict_counts["Total"] = totals


    verdict_counts[valid_verdicts] = verdict_counts[valid_verdicts].astype("string")

    if "borderlines" in totals.index:
        verdict_counts.loc["borderlines", valid_verdicts] = "—"
        verdict_counts.loc["borderlines", "Total"] = totals["borderlines"]


    verdict_counts = verdict_counts.reset_index().rename(
        columns={"category": "Category"}
    )
    verdict_counts = verdict_counts.sort_values("Category")


    # create table
    print("\\begin{table}[H]")
    print("\\centering")
    print("\\begin{tabular}{lcccc}")
    print("\\hline")
    print("Category & True & False & Half True & Total \\\\")
    print("\\hline")

    for _, r in verdict_counts.iterrows():
        print(
            f"{r['Category']} & {r['True']} & {r['False']} & "
            f"{r['Half True']} & {r['Total']} \\\\"
        )

    print("\\hline")
    print("\\end{tabular}")
    print("\\end{table}")



    categories = verdict_counts["Category"].tolist()
    
    return verdict_counts, categories


In [29]:
#df_druid = pd.read_csv("../data/DruidData/flatten_claims.csv", sep="\t")
# cnt of evidences
summary_claims_all, category_universe = summarize_claims_by_category_and_verdict("../data/flatten_claims.csv")
summary_claims_all


\begin{table}[H]
\centering
\begin{tabular}{lcccc}
\hline
Category & True & False & Half True & Total \\
\hline
borderlines & — & — & — & 197.0 \\
checkyourfact & 8 & 199 & 0 & 207.0 \\
climatefeedback & 0 & 5 & 0 & 5.0 \\
factcheckni & 43 & 39 & 0 & 82.0 \\
factly & 35 & 111 & 11 & 157.0 \\
healthfeedback & 0 & 5 & 0 & 5.0 \\
politifact & 52 & 101 & 36 & 189.0 \\
sciencefeedback & 13 & 167 & 6 & 186.0 \\
srilankafactcrescendo & 0 & 143 & 0 & 143.0 \\
\hline
\end{tabular}
\end{table}


factcheck_verdict,Category,True,False,Half True,Total
8,borderlines,—,—,—,197.0
0,checkyourfact,8,199,0,207.0
1,climatefeedback,0,5,0,5.0
2,factcheckni,43,39,0,82.0
3,factly,35,111,11,157.0
4,healthfeedback,0,5,0,5.0
5,politifact,52,101,36,189.0
6,sciencefeedback,13,167,6,186.0
7,srilankafactcrescendo,0,143,0,143.0


In [31]:
category_universe

['borderlines',
 'checkyourfact',
 'climatefeedback',
 'factcheckni',
 'factly',
 'healthfeedback',
 'politifact',
 'sciencefeedback',
 'srilankafactcrescendo']

In [11]:
def summarize_claim_cnt_from_split_factcheck_json(items, modes, categories):
    df = pd.DataFrame(items)

    df["category"] = df["claim_id"].str.split(r"[-_]").str[0]
    df["mode"] = df["mode"].astype(str).str.strip()

    summary = (
        df[df["mode"].isin(modes)]
        .groupby(["category", "mode"])
        .size()
        .unstack(fill_value=0)
        .reindex(index=categories, columns=modes, fill_value=0)
    )


    totals = df.groupby("category").size()
    summary["Total"] = totals.reindex(categories, fill_value=0)


    summary[modes] = summary[modes].astype("string")

    #if "borderlines" in categories:
    #    summary.loc["borderlines", modes] = "—"
    #    summary.loc["borderlines", "Total"] = totals.get("borderlines", 0)


    summary = summary.reset_index().rename(columns={"category": "Category"})
    summary = summary.sort_values("Category")

    
    total_row = {}
    
    for mode in modes:
        # convert back to numeric temporarily to sum
        total_row[mode] = (
            summary[mode]
            .astype("Int64")
            .sum()
        )
    
    total_row["Total"] = summary["Total"].sum()
    total_row["Category"] = "Total"
    summary.loc["Total"] = total_row



    print("\\begin{table}[H]")
    print("\\centering")
    print("\\begin{tabular}{lcccc}")
    print("\\hline")
    print(f"\\Category & {modes[0]} & {modes[1]} & {modes[2]} & Total")
    print("\\hline")

    for _, r in summary.iterrows():
        print(
            f"{r['Category']} & {r[modes[0]]} & {r[modes[1]]} & "
            f"{r[modes[2]]} & {r['Total']} \\\\"
        )

    print("\\hline")
    print("\\end{tabular}")
    print("\\end{table}")


    return summary





In [13]:

model = "qwen"
modes = ["2support", "1refute", "mix"]
#factchecks = ["true", "false", "half_true"]
factcheck = "true"
BASE_DIR = Path("../data/split_by_verdict")
all_items = []

for mode in modes:
    json_path = (BASE_DIR / factcheck/ f"{model}_{mode}_{factcheck}_bullet_first_result.json")

    if not json_path.exists():
        print(f"Missing file: {json_path}")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        items = json.load(f)

    all_items.extend(items)

    if not all_items:
        print(f"No data for mode = {mode}")
        continue

true_summary_table = summarize_claim_cnt_from_split_factcheck_json(all_items, modes, category_universe)
print(f"\n=== {model} | {factcheck } ===")
display(true_summary_table)

\begin{table}[H]
\centering
\begin{tabular}{lcccc}
\hline
\Category & 2support & 1refute & mix & Total
\hline
borderlines & 0 & 0 & 0 & 0 \\
checkyourfact & 5 & 1 & 1 & 7 \\
climatefeedback & 0 & 0 & 0 & 0 \\
factcheckni & 29 & 2 & 2 & 33 \\
factly & 26 & 5 & 3 & 34 \\
healthfeedback & 0 & 0 & 0 & 0 \\
politifact & 38 & 3 & 2 & 43 \\
sciencefeedback & 12 & 0 & 0 & 12 \\
srilankafactcrescendo & 0 & 0 & 0 & 0 \\
Total & 110 & 11 & 8 & 129 \\
\hline
\end{tabular}
\end{table}

=== qwen | true ===


mode,Category,2support,1refute,mix,Total
0,borderlines,0,0,0,0
1,checkyourfact,5,1,1,7
2,climatefeedback,0,0,0,0
3,factcheckni,29,2,2,33
4,factly,26,5,3,34
5,healthfeedback,0,0,0,0
6,politifact,38,3,2,43
7,sciencefeedback,12,0,0,12
8,srilankafactcrescendo,0,0,0,0
Total,Total,110,11,8,129


In [15]:
model = "qwen"
modes = ["2support", "2refute", "mix"]
#factchecks = ["true", "false", "half_true"]
factcheck = "false"
BASE_DIR = Path("../data/split_by_verdict")
all_items = []

for mode in modes:
    json_path = (BASE_DIR / factcheck/ f"{model}_{mode}_{factcheck}_bullet_first_result.json")

    if not json_path.exists():
        print(f"Missing file: {json_path}")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        items = json.load(f)

    all_items.extend(items)

    if not all_items:
        print(f"No data for mode = {mode}")
        continue

false_summary_table = summarize_claim_cnt_from_split_factcheck_json(all_items, modes, category_universe)
print(f"\n=== {model} | {factcheck } ===")
display(false_summary_table)

\begin{table}[H]
\centering
\begin{tabular}{lcccc}
\hline
\Category & 2support & 2refute & mix & Total
\hline
borderlines & 0 & 0 & 0 & 0 \\
checkyourfact & 19 & 111 & 55 & 185 \\
climatefeedback & 0 & 3 & 0 & 3 \\
factcheckni & 1 & 26 & 8 & 35 \\
factly & 16 & 75 & 27 & 118 \\
healthfeedback & 0 & 4 & 1 & 5 \\
politifact & 5 & 77 & 20 & 102 \\
sciencefeedback & 3 & 124 & 26 & 153 \\
srilankafactcrescendo & 12 & 100 & 24 & 136 \\
Total & 56 & 520 & 161 & 737 \\
\hline
\end{tabular}
\end{table}

=== qwen | false ===


mode,Category,2support,2refute,mix,Total
0,borderlines,0,0,0,0
1,checkyourfact,19,111,55,185
2,climatefeedback,0,3,0,3
3,factcheckni,1,26,8,35
4,factly,16,75,27,118
5,healthfeedback,0,4,1,5
6,politifact,5,77,20,102
7,sciencefeedback,3,124,26,153
8,srilankafactcrescendo,12,100,24,136
Total,Total,56,520,161,737


In [17]:
model = "qwen"
modes = ["2support", "2refute", "mix"]
#factchecks = ["true", "false", "half_true"]
factcheck = "half_true"
BASE_DIR = Path("../data/split_by_verdict")
all_items = []

for mode in modes:
    json_path = (BASE_DIR / factcheck/ f"{model}_{mode}_{factcheck}_bullet_first_result.json")

    if not json_path.exists():
        print(f"Missing file: {json_path}")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        items = json.load(f)

    all_items.extend(items)

    if not all_items:
        print(f"No data for mode = {mode}")
        continue

half_true_summary_table = summarize_claim_cnt_from_split_factcheck_json(all_items, modes, category_universe)
print(f"\n=== {model} | {factcheck} ===")
display(half_true_summary_table)

\begin{table}[H]
\centering
\begin{tabular}{lcccc}
\hline
\Category & 2support & 2refute & mix & Total
\hline
borderlines & 0 & 0 & 0 & 0 \\
checkyourfact & 0 & 0 & 0 & 0 \\
climatefeedback & 0 & 0 & 0 & 0 \\
factcheckni & 0 & 0 & 0 & 0 \\
factly & 7 & 0 & 0 & 7 \\
healthfeedback & 0 & 0 & 0 & 0 \\
politifact & 17 & 2 & 7 & 26 \\
sciencefeedback & 1 & 2 & 0 & 3 \\
srilankafactcrescendo & 0 & 0 & 0 & 0 \\
Total & 25 & 4 & 7 & 36 \\
\hline
\end{tabular}
\end{table}

=== qwen | half_true ===


mode,Category,2support,2refute,mix,Total
0,borderlines,0,0,0,0
1,checkyourfact,0,0,0,0
2,climatefeedback,0,0,0,0
3,factcheckni,0,0,0,0
4,factly,7,0,0,7
5,healthfeedback,0,0,0,0
6,politifact,17,2,7,26
7,sciencefeedback,1,2,0,3
8,srilankafactcrescendo,0,0,0,0
Total,Total,25,4,7,36
